# Places in Moscow to open Italian restaurant

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for Italian restaurant in Moscow, Russia.

Since there are lots of restaurants in Moscow we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Italian restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing districts based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the district (any type of restaurant)
* number of and distance to Italian restaurants in the district, if any
* distance of district from city center

We decided to use regularly spaced grid of locations, centered around city center, to define city districts.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Moscow center will be obtained using **Google Maps API geocoding** of well known Moscow location (Kremlin)

### District Candidates

Let's create latitude & longitude coordinates for centroids of our candidate districts. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Moscow city center.

Let's first find the latitude & longitude of Moscow city center, using specific, well known address and Google Maps geocoding API.

In [51]:
google_api_key="" # Google API key (removed for safety)

In [4]:
import requests

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Kremlin, Moscow'
moscow_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address,moscow_center))

Coordinate of Kremlin, Moscow: [55.7520233, 37.6174994]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Kremlin. Our districts will be defined as circular areas with a radius of 300 meters, so our district centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [14]:
#!pip3 install shapely
#!pip3 install pyproj
#!conda install geos

Solving environment: ...working... done

# All requested packages already installed.



In [29]:
import shapely.geometry

import pyproj

import math

from pyproj import Transformer
from pyproj import CRS

def lonlat_to_xy(lon, lat):
    crs_latlon = CRS(proj='latlong',datum='WGS84')
    crs_xy = CRS(proj="utm", zone=33, datum='WGS84')
    transformer = Transformer.from_crs(crs_latlon, crs_xy)
    xy = transformer.transform(lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    crs_latlon = CRS(proj='latlong',datum='WGS84')
    crs_xy = CRS(proj="utm", zone=33, datum='WGS84')
    transformer = Transformer.from_crs(crs_xy, crs_latlon)
    lonlat = transformer.transform(x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Moscow center longitude={}, latitude={}'.format(moscow_center[1], moscow_center[0]))
x, y = lonlat_to_xy(moscow_center[1], moscow_center[0])
print('Moscow center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Moscow center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Moscow center longitude=37.6174994, latitude=55.7520233
Moscow center UTM X=1905280.7258497092, Y=6412722.831259564
Moscow center longitude=37.61749940000001, latitude=55.7520233


Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [30]:
moscow_center_x, moscow_center_y = lonlat_to_xy(moscow_center[1], moscow_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = moscow_center_x - 6000
x_step = 600
y_min = moscow_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(moscow_center_x, moscow_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate district centers generated.')

364 candidate district centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [31]:
#!pip install folium

import folium

In [32]:
map_moscow = folium.Map(location=moscow_center, zoom_start=13)
folium.Marker(moscow_center, popup='Kremlin').add_to(map_moscow)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_moscow) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_moscow)
    #folium.Marker([lat, lon]).add_to(map_moscow)
map_moscow

OK, we now have the coordinates of district's centers to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Kremlin. 

Let's now use Google Maps API to get approximate addresses of those locations.

In [33]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, moscow_center[0], moscow_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(moscow_center[0], moscow_center[1], addr))

Reverse geocoding check
-----------------------
Address of [55.7520233, 37.6174994] is: Troitskaya Ulitsa, Moskva, Russia, 109012


In [34]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', Russia', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [35]:
addresses[150:170]

['Talalikhina Ulitsa, 28, Moskva, 109316',
 'Mikhaylovskiy Proyezd, 3 строение 15, Moskva, Moscow Oblast, 109029',
 'Skotoprogonnaya Ulitsa, 35 строение 1, Moskva, 109029',
 'Ulitsa Podvoyskogo, 5/19 строение 1, Moskva, 123317',
 'Ulitsa Litvina-Sedogo, 2, Moskva, 123317',
 'Ulitsa Anatoliya Zhivova, 5, Moskva, 123100',
 "Bol'shoy Trekhgornyy Pereulok, 1/26 строение 2, Moskva, 123022",
 'Rochdelskaya St, 11/5, Moskva, 123022',
 'Bolshoy Devyatinsky Ln, 15, Moskva, 121099',
 'New Arbat Ave, 24, Moskva, 121099',
 'Arbat St, 13, Moskva, 119019',
 'Malyy Znamenskiy Ln, 11/11 строение 1, Moskva, 119019',
 'Sofiyskaya Embankment, дом 11, Moskva, 119072',
 'Balchug St, Moskva, 115035',
 'Sredniy Ovchinnikovskiy Pereulok, 16, Moskva, 115184',
 'Kosmodamianskaya Naberezhnaya, 24 строение 1, Moskva, 115035',
 'Goncharnaya Ulitsa, 38, Moskva, 115172',
 'Marksistskaya St, 16, Moskva, 109147',
 'Marksistskaya St, 9 корпус 2, Moskva, 109147',
 'Talalikhina Ulitsa, 16, Moskva, 109316']

Looking good. Let's now place all this into a Pandas dataframe.

In [36]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"Unnamed Road, Москва, 119334",55.709742,37.562144,1.903481e+06,6.407007e+06,5992.495307
1,"Ulitsa Kosygina, 4 корпус 11, Moskva, 119334",55.708033,37.570961,1.904081e+06,6.407007e+06,5840.376700
2,"Leninskiy Prospekt, 41/2, Moskva, 119334",55.706323,37.579777,1.904681e+06,6.407007e+06,5747.173218
3,"Ulitsa Vavilova, 5Б строение 5, Moskva, 119334",55.704612,37.588592,1.905281e+06,6.407007e+06,5715.767665
4,"5-th Donskoy Passage, 21А строение 2, Moskva, ...",55.702901,37.597407,1.905881e+06,6.407007e+06,5747.173218
5,"Загородное ш., владение 2Б, Moskva, 115419",55.701189,37.606220,1.906481e+06,6.407007e+06,5840.376700
6,"Zagorodnoye Shosse, 1 корпус 2 строение 2, Mos...",55.699477,37.615032,1.907081e+06,6.407007e+06,5992.495307
7,"Unnamed Road, Moskva, 119270",55.716617,37.551537,1.902581e+06,6.407527e+06,5855.766389
8,"ul. Luzhniki, 24 строение 8, Moskva, 119270",55.714908,37.560356,1.903181e+06,6.407527e+06,5604.462508
9,"Luzhniki Embankment, 6, стр. 1, Moskva, 119270",55.713199,37.569175,1.903781e+06,6.407527e+06,5408.326913


...and let's now save/persist this data into local file.

In [37]:
df_locations.to_pickle('./locations.pkl')    

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Italian restaurant' category, as we need info on Italian restaurants in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.

In [52]:
foursquare_client_id = '' # Foursquare ID (removed for safity)
foursquare_client_secret = '' # Foursquare Secret (removed for safity)

In [42]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

italian_restaurant_categories = ['4bf58dd8d48988d110941735','55a5a1ebe4b013909087cbb6','55a5a1ebe4b013909087cb7c',
                                 '55a5a1ebe4b013909087cba7','55a5a1ebe4b013909087cba1','55a5a1ebe4b013909087cba4',
                                 '55a5a1ebe4b013909087cb95','55a5a1ebe4b013909087cb89','55a5a1ebe4b013909087cb9b',
                                 '55a5a1ebe4b013909087cb98','55a5a1ebe4b013909087cbbf','55a5a1ebe4b013909087cb79',
                                 '55a5a1ebe4b013909087cbb0','55a5a1ebe4b013909087cbb3','55a5a1ebe4b013909087cb74',
                                 '55a5a1ebe4b013909087cbaa','55a5a1ebe4b013909087cb83','55a5a1ebe4b013909087cb8c',
                                 '55a5a1ebe4b013909087cb92','55a5a1ebe4b013909087cb8f','55a5a1ebe4b013909087cb86',
                                 '55a5a1ebe4b013909087cbb9','55a5a1ebe4b013909087cb7f','55a5a1ebe4b013909087cbbc',
                                 '55a5a1ebe4b013909087cb9e','55a5a1ebe4b013909087cbc2','55a5a1ebe4b013909087cbad']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [43]:
# Let's now go over our district locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    italian_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=350, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_italian = is_restaurant(venue_categories, specific_filter=italian_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_italian, x, y)
                if venue_distance<=300:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_italian:
                    italian_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, italian_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
italian_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('italian_restaurants_350.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, italian_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('italian_restaurants_350.pkl', 'wb') as f:
        pickle.dump(italian_restaurants, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [44]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 1670
Total number of Italian restaurants: 171
Percentage of Italian restaurants: 10.24%
Average number of restaurants in neighborhood: 4.271978021978022


In [45]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4d090b02790fa093c1a0c673', 'Илларион', 55.707295922495504, 37.57271761523504, 'ул. Косыгина, 5 (ул. Академика Зелинского), 119334, Москва, Россия', 137, False, 1904214.9595365492, 6406964.318861007)
('4c2b2a948abca5930fafff1f', 'Анджело / Angelo', 55.70339521952314, 37.58032118164971, 'ул. 60-летия Октября, 5 (ул. Вавилова), 119334, Москва, Россия', 327, True, 1904822.3045026094, 6406702.791750991)
('4fb7e42ae4b0d5ed25a15546', 'Желтое Море японский ресторан', 55.70567815197633, 37.58138856973665, 'Москва, Россия', 123, False, 1904802.7150885, 6406971.241812426)
('5b1ab242b1538e002c3a269e', 'Sticks', 55.707422, 37.589481, '119334, Москва, Россия', 311, False, 1905230.7070696978, 6407328.545284792)
('55dc2479498e7f12a6982bde', 'Всё хоккей!', 55.704137944260765, 37.59372186120669, 'Автозаводская, 23, Москва, Россия', 269, False, 1905610.5746430587, 6407063.253876937)
('4f72e1aae4b062fe7c1d9374', 'Rice', 55.704778637931454, 37.5964961161850

In [46]:
print('List of Italian restaurants')
print('---------------------------')
for r in list(italian_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(italian_restaurants))

List of Italian restaurants
---------------------------
('4c2b2a948abca5930fafff1f', 'Анджело / Angelo', 55.70339521952314, 37.58032118164971, 'ул. 60-летия Октября, 5 (ул. Вавилова), 119334, Москва, Россия', 327, True, 1904822.3045026094, 6406702.791750991)
('51347e1be4b0872514a9e658', 'Ньокки', 55.71038805237878, 37.62177993574395, 'Большая Тульская ул., 2, Москва, Россия', 271, True, 1907086.5621759046, 6408324.620579807)
('5c0bb75425ecca002c974698', 'Pasta Cup & Pinsa', 55.704598, 37.639112, 'Автозаводская ул., 18, 115280, Москва, Россия', 75, True, 1908356.4113605088, 6408063.849212115)
('5aef1198b8fd9d002c881b63', 'Il Patio', 55.704935, 37.640758, 'Автозаводская ул., 18, 115280, Москва, Россия', 180, True, 1908444.0773889103, 6408134.698114456)
('51b850eb498e00e0dddc488e', 'Qustoso Cafe', 55.725050456469496, 37.56389369737514, 'ул. 10-летия Октября, 11 (ул. Усачева), Москва, Россия', 200, True, 1903020.5548372418, 6408694.027461919)
('54957eef498e9c88a80a9fd2', 'Fiorella Pasta Fr

In [47]:
print('Restaurants around location')
print('---------------------------')
for i in range(100, 110):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('Restaurants around location {}: {}'.format(i+1, names))

Restaurants around location
---------------------------
Restaurants around location 101: Mon bon, Marketplace, Sumosan, Урюк, Russkyi, Mammina, Чайхона № 1, Mukuzani
Restaurants around location 102: Modus Restaurant
Restaurants around location 103: Семифреддо, Корчма Тарас Бульба, Кампус, Grange
Restaurants around location 104: Accenti, Жеральдин, Тифлисский дворик, Плов, g'Arbuz, КасБар, Brasserie Lambic, Консул
Restaurants around location 105: Piazza Italiana
Restaurants around location 106: Pancho Villa, Ichiban Boshi (Ичибан Боши), ВьетКафе
Restaurants around location 107: Чайхона № 1, Brasserie Lambic, Моремания, Кафе БайХан при Посольстве Кыргызстана, Сомбреро, Ресторан Полянка
Restaurants around location 108: Oblomov (Обломов), Ача-чача, Mojo, Кимчи, Якитория
Restaurants around location 109: 
Restaurants around location 110: Boston Seafood & Bar, Torro Grill, Суши Wok, Столовка, Колхида, Канэ, сыр и вино, Кожевническая Ул.11/13 Стр.1


Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [50]:
map_moscow = folium.Map(location=moscow_center, zoom_start=13)
folium.Marker(moscow_center, popup='Kremlin').add_to(map_moscow)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_moscow)
map_moscow

Looking good. So now we have all the restaurants in area within few kilometers from Kremlin, and we know which ones are Italian restaurants! We also know which restaurants exactly are in vicinity of every district candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!